# Data Munging

Voy a analizar y validar un set de datos sin usar Pandas porque no está instalado en Cambrian. Abajo dejo algunos links para ver con Pandas. 

Leer archivos CSVs gigantes:

* https://stackoverflow.com/questions/17444679/reading-a-huge-csv-file (usa las **[generator functions](https://www.programiz.com/python-programming/generator)** )

Otros:
* https://stackoverflow.com/questions/17006708/how-to-process-huge-csv-file-into-python
* https://stackoverflow.com/questions/42900757/sequentially-read-huge-csv-file-in-python
* https://stackoverflow.com/questions/28951172/improving-efficiency-of-reading-a-large-csv-file
* https://stackoverflow.com/questions/9087039/most-efficient-way-to-parse-a-large-csv-in-python

Con Pandas:
* Data Munging with Python, SQL and Excel: https://blogs.library.ucsf.edu/ckm/2016/09/27/data-munging-with-python-sql-and-excel/

Ver tambien el notebook con el tutorial de Pandas


## Detectar separador

Para buscar el separador puedo ver las 10 primeras líneas de un archvio ('head') y contar:

* TABS =  grep -o -P "\t" sc.csv|wc -l
* PYC = grep -o ';' sc.csv|wc -l
* COM = grep -o ',' sc.csv|wc -l
* ESP = grep -o ' ' sc.csv|wc -l

El que obtenga el mayor número se considerará el separador.

In [64]:
#Usar: import os
# PYC = os.popen('head %s | grep -o -P \"\\t\"|wc -l' %archivo).read()
# die en python: #sys.exit()

#Supiendo que ya obtuve de los comandos anteriores las cantidades armo un diccionario y obtengo el mayor.
PYC = 30
COM = 5
ESP = 5
TABS = 2
sep = {';':PYC, ',':COM,' ':ESP,'\t':TABS}

#Esta funcion obtiene el key con el mayor valor.
#separador=max(sep,key=lambda k: sep[k])
separador=max(sep,key=sep.get)

print ("El separador es:'%s'" %separador)

# De: https://stackoverflow.com/questions/42174367/how-can-i-determine-the-number-of-fields-in-a-csv-from-the-shell
# y: https://stackoverflow.com/questions/450799/shell-command-to-sum-integers-one-per-line
#Número de campos: head -n 10 sc.csv | awk -F";" '{print NF}'
#Para que sume todas las lineas: CANTIDAD_CAMPOS =   head -n 10 sc.csv | awk -F";" '{print NF}'|awk '{s+=$1} END {print s}'
#Cuento 10 líneas del archivo. Si CANTIDAD_CAMPOS % 0 es distinto de cero => la cantida de campos está mal = > TERMNAR EJ

CANTIDAD_CAMPOS = 60

if (CANTIDAD_CAMPOS%10):
    print ("El modulo <> 0 - TERMINAR EJECUCION DEL PROGRAMA CON ERROR")
else:
    CANTIDAD_CAMPOS = int(CANTIDAD_CAMPOS/10)
    print ("Modulo = 0 - CAMPOS = ", CANTIDAD_CAMPOS)

El separador es:';'
Modulo = 0 - CAMPOS =  6


In [41]:
#Leer un csv con Pandas
#df = pd.read_csv(archivo") 
#df.head(10)

#Abrir y leer csv sin Pandas (como Cambrian no tiene Pandas por ahora lo tengo que resolver así)
#import csv
nombre_archivo = "scentia_test.csv"
path='C:/Users/F/Dropbox/Personal/CienciaDeDatos/98_Miscelaneos/pydata101/TutorialPython/datasets/'
archivo=path + nombre_archivo

#Abrir el archivo: https://docs.python.org/2/tutorial/inputoutput.html#reading-and-writing-files
#           r -> read (default si no se aclara)
#           w -> write (OJO: archivo con el mismo nombre se borra)
#           a -> append (agregar más datos al final de un archivo existente)
#           r+ -> read/write
#           b -> binary. Usar siempre en Windows para archivos binarios. Si un archivo binario se abre como texto en windows puede corromperse.
#           r+b -> read write binary.
# OJO: En windows usar siempre el modo 'b' para no corromper archivos binarios. En Linux es indiferente. 
#Por lo tanto, para no tener problemas de plataforma USAR SIEMPRE 'b'

csv_file = open(archivo, 'r')

#FALTA: buscar el separador del archivo. Ver cuantas 
c=0
for row in csv_file:    
    row=row.rstrip()
    print ('Fila %s: %s' %(c,row)); c += 1    #rstrip == chomp
    #COM = row.count(','); PYC = row.count(';');  TABS = row.count('\t'); ESP = row.count(' ')
    #print("Separadores:\n\tComa: %s\n\tPunto y coma: %s\n\tTabs: %s\n\tESP: %s" %(COM,PYC,TABS,ESP))
    #col = row.decode('ascii').split(';')    #El decode se usa porque lo abrí como binario, sino hay que abrir como texto (sin la 'b')
    #col = row.split(';')
    col = row.split(separador)
    #print(type(col))
    print("Cantidad de campos: %s\n" %len(col))
    #print (col[0])
    
csv_file.close()           #Cerrar el archivo cuando no se use mas

Fila 0: FECHA;HORA;EAN;MONTO_UNITARIO;CANTIDAD;MONTO_TOTAL
Cantidad de campos: 6

Fila 1: 2017-09-05 00000000;	203405;0001;  3,56;1;3,56
Cantidad de campos: 6

Fila 2: 2017-09-05 00000000;203405;0001;4,05;2;8,1
Cantidad de campos: 6

Fila 3: 2017-15-05 00000000;203405;0001;4,05;2;8,1
Cantidad de campos: 6

Fila 4: 2017-09-05 00000000;283405;0001;3,56;1;3,56
Cantidad de campos: 6

Fila 5: 2017-09-05 00000000;;0001;3,56;1;3,56
Cantidad de campos: 6



In [96]:
#Abrir y leer csv sin Pandas (como Cambrian no tiene Pandas por ahora lo tengo que resolver así)
import time
start_time = time.time()

nombre_archivo = "scentia_test.csv"
path='C:/Users/F/Dropbox/Personal/CienciaDeDatos/98_Miscelaneos/pydata101/TutorialPython/datasets/'
archivo=path + nombre_archivo

csv_file = open(archivo, 'r')

c=0
for row in csv_file:    
    row=row.rstrip()                           #rstrip == chomp
    #print ('Fila %s: %s' %(c,row)); 
    c += 1    
    col = row.split(separador)
    #print("Cantidad de campos: %s\n" %len(col))
    if (CANTIDAD_CAMPOS != len(col)):
        print("Linea %s: Cantidad de campos incorrecta - MANDAR LINEA A LOG DE ERRORES" %c)
    else:
        print("Linea %s: Cantidad de campos correcta - Seguir verificando" %c)
        FECHA = col[0]
        HORA = col[1]
        EAN = col[2]
        MONTO_UNITARIO = col[3]
        
csv_file.close()           #Cerrar el archivo cuando no se use mas

end_time = time.time()
print("--- Tiempo de ejecución: %s seconds ---" % (end_time - start_time))

Linea 1: Cantidad de campos correcta - Seguir verificando
Linea 2: Cantidad de campos correcta - Seguir verificando
Linea 3: Cantidad de campos correcta - Seguir verificando
Linea 4: Cantidad de campos correcta - Seguir verificando
Linea 5: Cantidad de campos incorrecta - MANDAR LINEA A LOG DE ERRORES
Linea 6: Cantidad de campos correcta - Seguir verificando
--- Tiempo de ejecución: 0.001953125 seconds ---
